In [44]:
import pandas as pd
import numpy as np 
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from os import system 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from random import randint
import plotly.plotly as py
import plotly.graph_objs as go
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
py.sign_in('kaushik.316', '86eaqb1c8w')

### Data Format

In [45]:
project_df = pd.read_csv('../Data/CONSOLIDATED_DCB_DATA.csv')
project_df = project_df.drop(project_df.columns[[0]],axis=1)
project_df = project_df[np.isfinite(project_df['Chg_from_50davg'])]
project_df.count()

Ticker             425
Chg_from_Hi        425
Chg_from_Lo        425
Chg_from_50davg    425
Short_Ratio        425
Is_Dead_Cat        425
dtype: int64

In [46]:
project_df.head()

,Ticker,Chg_from_Hi,Chg_from_Lo,Chg_from_50davg,Short_Ratio,Is_Dead_Cat
0,USCR,-7.649554,72.938921,1.453999,4.083659,1
1,SRPT,-45.872218,110.614525,23.350694,3.639042,1
2,SPWR,-49.756256,14.603410,-8.774085,7.637297,1
3,CMA,-23.008850,29.013540,-8.893328,3.340925,1
4,AKS,-8.971963,166.120219,13.191348,2.565646,1


### Visualizations

In [47]:
DCB_df = project_df[project_df['Is_Dead_Cat']==1]
nonDCB_df = project_df[project_df['Is_Dead_Cat']==0]

#### Distribution

In [48]:
from plotly.tools import FigureFactory as FF

x1 = DCB_df['Chg_from_Hi']
x2 = DCB_df['Chg_from_Lo'] 
x3 = DCB_df['Chg_from_50davg']
x4 = DCB_df['Short_Ratio']

hist_data = [x1,x2,x3,x4]
group_labels = ['Chg_from_Hi','Chg_from_Lo','Chg_from_50davg','Short_Ratio']
fig = FF.create_distplot(hist_data, group_labels, show_hist=False)

py.iplot(fig, filename='DCB Distplot with Multiple Datasets', validate=False)

#### Mean Comparison

In [49]:
mean_cols = ['Chg_from_Hi','Chg_from_Lo','Chg_from_50davg','Short_Ratio']
DCB_means = DCB_df[mean_cols].mean()
nonDCB_means = nonDCB_df[mean_cols].mean()

x1 = go.Bar(
    x= mean_cols,
    y=DCB_means,
    name='DCB Stocks'
)

x2 = go.Bar(
    x=mean_cols,
    y=nonDCB_means,
    name='nonDCB stocks'
)

data = [x1,x2]
layout = go.Layout(
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='DCB mean grouped-bar', title="Averages")

#### Median Comparison

In [50]:
median_cols = ['Chg_from_Hi','Chg_from_Lo','Chg_from_50davg','Short_Ratio']
DCB_medians = DCB_df[median_cols].median()
nonDCB_medians = nonDCB_df[median_cols].median()

m1 = go.Bar(
    x= median_cols,
    y=DCB_medians,
    name='nonDCB Stocks'
)

m2 = go.Bar(
    x=median_cols,
    y=nonDCB_medians,
    name='nonDCB stocks'
)

data = [m1,m2]
layout = go.Layout(
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='DCB median grouped-bar')

### Creating the Training and Test Dataset

In [51]:
def make_noise(x):
    rand = randint(0,1)
    if rand == 0:
        return x * .95
    else:
        return x * 1.05
    

In [52]:
# Oversampling - create copies of less represented class. 
DCB_df_copy = DCB_df
DCB_df_copy.head()

,Ticker,Chg_from_Hi,Chg_from_Lo,Chg_from_50davg,Short_Ratio,Is_Dead_Cat
0,USCR,-7.649554,72.938921,1.453999,4.083659,1
1,SRPT,-45.872218,110.614525,23.350694,3.639042,1
2,SPWR,-49.756256,14.603410,-8.774085,7.637297,1
3,CMA,-23.008850,29.013540,-8.893328,3.340925,1
4,AKS,-8.971963,166.120219,13.191348,2.565646,1


In [53]:
# Add a little noise to the copied values
for col in DCB_df_copy.columns[1:-1]:
    DCB_df_copy[col] = DCB_df_copy[col].apply(make_noise)

DCB_df_copy.head()

/Users/kaushikvisvanathan/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,Ticker,Chg_from_Hi,Chg_from_Lo,Chg_from_50davg,Short_Ratio,Is_Dead_Cat
0,USCR,-7.267076,76.585867,1.381299,4.287842,1
1,SRPT,-43.578607,116.145251,24.518229,3.457090,1
2,SPWR,-52.244069,13.873239,-9.212789,8.019161,1
3,CMA,-21.858407,30.464217,-8.448662,3.507972,1
4,AKS,-9.420561,157.814208,13.850915,2.693928,1


In [54]:
project_df = pd.concat([project_df, DCB_df_copy])
print project_df.Is_Dead_Cat.value_counts()

0    363
1    124
Name: Is_Dead_Cat, dtype: int64


#### Training Dataset

In [55]:
# shuffle data
project_df = project_df.sample(frac=1)

X = project_df.iloc[:, 1:-1]
y = project_df.iloc[:, -1:]

In [56]:
x_traindf, x_testdf, y_traindf, y_testdf = train_test_split(X, y, test_size=0.25, random_state=42)

In [57]:
x_traindf.count()

Chg_from_Hi        365
Chg_from_Lo        365
Chg_from_50davg    365
Short_Ratio        365
dtype: int64

In [58]:
y_traindf['Is_Dead_Cat'].value_counts()

0    269
1     96
Name: Is_Dead_Cat, dtype: int64

#### Test Dataset

In [59]:
x_testdf.count()

Chg_from_Hi        122
Chg_from_Lo        122
Chg_from_50davg    122
Short_Ratio        122
dtype: int64

In [60]:
y_testdf['Is_Dead_Cat'].value_counts()

0    94
1    28
Name: Is_Dead_Cat, dtype: int64

In [61]:
# Convert to numpy array for uniformity
x_train = x_traindf.as_matrix()
y_train = y_traindf.as_matrix()
x_test = x_testdf.as_matrix()
y_test = y_testdf.as_matrix()

# reshape y
y_train = np.reshape(y_train, (len(y_train), ))
y_test = np.reshape(y_test, (len(y_test), ))

### Support Vector Machine 

In [62]:
# Parameter grid for grid search
# Omit C values from the grid as class_weight param does this automatically for us
svm_parameters = {'kernel': ['linear', 'rbf'],
			  	  'degree': [1,3,5],
				  'gamma': [1/(len(x_train)), 1e-1]}

cv = StratifiedShuffleSplit(n_splits=10, test_size=0.2)
svm_clf = SVC(class_weight='balanced', probability=True)

In [63]:
grid = GridSearchCV(svm_clf, param_grid=svm_parameters, cv=cv)
grid.fit(x_train, y_train)
print("The best parameters are %s with a score of %0.2f" % (grid.best_params_, grid.best_score_))

grid_score = grid.score(x_test, y_test)
print "Grid search's score on new test data was {}".format(grid_score)

The best parameters are {'kernel': 'rbf', 'gamma': 0.1, 'degree': 1} with a score of 0.80
Grid search's score on new test data was 0.868852459016


In [64]:
# Map from predictions to probabilities based on set threshold
def map_predictions(x):
    threshold = 0.4
    if x > threshold:
        return 1
    else:
        return 0

In [68]:
# Make predictions on the model
x_testdf['predicted_prob'] = grid.predict_proba(x_test)[:, 1]
x_testdf['predictions'] = x_testdf['predicted_prob'].apply(map_predictions)
x_testdf['actuals'] = y_test
x_testdf.head()

/Users/kaushikvisvanathan/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/kaushikvisvanathan/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/kaushikvisvanathan/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

,Chg_from_Hi,Chg_from_Lo,Chg_from_50davg,Short_Ratio,predicted_prob,predictions,actuals
373,-4.00,128.43,8.42,6.00,0.228293,0,0
396,-13.57,9.26,-1.68,8.89,0.091695,0,0
214,-36.88,48.95,-4.37,1.91,0.235052,0,0
167,-0.68,65.63,4.76,9.35,0.235177,0,0
258,-65.04,39.64,-1.09,19.97,0.236075,0,0


#### Accuracy measure

Since occurrences of dead cat bounces are rare, our data is unbalanced. To ensure that our model is not simply predicting a zero every time, we can examine the precision, which should be high. A high precision indicates that the number of true positives is high relative to the number of false positives.

In [70]:
grid_score = grid.score(x_test, y_test)
grid_precision = precision_score(x_testdf["predictions"], x_testdf["actuals"])
grid_recall = recall_score(x_testdf["predictions"], x_testdf["actuals"])

print "Model accuracy is {}".format(grid_score)
print "Model precision is {}".format(grid_precision)
print "Model recall is {}".format(grid_recall)

Model accuracy is 0.868852459016
Model precision is 0.714285714286
Model recall is 0.909090909091


We can see that the precision of our model is lower than overall accuracy, though the overall precision is not bad. As shown in the heat map below, the number of false positives and false negatives predicted is acceptable.

#### Confusion Matrix

In [71]:
y_true = x_testdf["actuals"]
y_pred = x_testdf["predictions"] 
matrix = confusion_matrix(y_true, y_pred)


print "True Negatives: %s" % (matrix[0][0])
print "False Positives: %s" % (matrix[0][1])
print "False Negatives: %s" % (matrix[1][0])
print "True Positives: %s" % (matrix[1][1])

True Negatives: 92
False Positives: 2
False Negatives: 8
True Positives: 20


In [83]:
matrixplot = [
    go.Heatmap(
        z= [[matrix[1][0], matrix[1][1]],[matrix[0][0], matrix[0][1]]],
        x=['Predicted No', 'Predicted Yes'],
        y=['Actual Yes', 'Actual No'],
    )
]
py.iplot(matrixplot, filename='labelled-heatmap')


### Decision Tree Model

In [75]:
tree_parameters = {'criterion': ['gini', 'entropy'],
			  	  'max_depth': [2,4,6],}

cv = StratifiedShuffleSplit(n_splits=10, test_size=0.2)
tree_clf = DecisionTreeClassifier(class_weight="balanced")

In [88]:
tree_grid = GridSearchCV(tree_clf, param_grid=tree_parameters, cv=cv)
tree_grid.fit(x_train, y_train)
print("The best parameters are %s with a score of %0.2f" % (tree_grid.best_params_, tree_grid.best_score_))

tree_grid_score = tree_grid.score(x_test, y_test)
print "Grid search's score on new test data was {}".format(tree_grid_score)

The best parameters are {'criterion': 'entropy', 'max_depth': 4} with a score of 0.82
Grid search's score on new test data was 0.795081967213


In [93]:
def visualize_tree(model):
    dotfile = open("tree.dot", 'w')
    export_graphviz(model, out_file = dotfile, feature_names = DCB_df.columns[1:5])
    dotfile.close()
    system("dot -Tpng tree.dot -o ../tree.png")

tree_clf.fit(x_train, y_train)
visualize_tree(tree_clf)

<img src="Tree.png">

In [77]:
# Make predictions on the model
x_testdf['t_predicted_prob'] = tree_grid.predict_proba(x_test)[:, 1]
x_testdf['t_predictions'] = x_testdf['predicted_prob'].apply(map_predictions)
x_testdf.head()

/Users/kaushikvisvanathan/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/kaushikvisvanathan/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,Chg_from_Hi,Chg_from_Lo,Chg_from_50davg,Short_Ratio,predicted_prob,predictions,actuals,t_predicted_prob,t_predictions
373,-4.00,128.43,8.42,6.00,0.228293,0,0,0.168020,0
396,-13.57,9.26,-1.68,8.89,0.091695,0,0,0.168020,0
214,-36.88,48.95,-4.37,1.91,0.235052,0,0,0.853034,0
167,-0.68,65.63,4.76,9.35,0.235177,0,0,0.168020,0
258,-65.04,39.64,-1.09,19.97,0.236075,0,0,0.168020,0


In [79]:
tree_grid_score = tree_grid.score(x_test, y_test)
tree_grid_precision = precision_score(x_testdf["t_predictions"], x_testdf["actuals"])
tree_grid_recall = recall_score(x_testdf["t_predictions"], x_testdf["actuals"])

print "Model accuracy is {}".format(grid_score)
print "Model precision is {}".format(grid_precision)
print "Model recall is {}".format(grid_recall)

Model accuracy is 0.868852459016
Model precision is 0.714285714286
Model recall is 0.909090909091


In [80]:
y_true = x_testdf["actuals"]
y_pred = x_testdf["t_predictions"] 
matrix = confusion_matrix(y_true, y_pred)

print "True Negatives: %s" % (matrix[0][0])
print "False Positives: %s" % (matrix[0][1])
print "False Negatives: %s" % (matrix[1][0])
print "True Positives: %s" % (matrix[1][1])

True Negatives: 92
False Positives: 2
False Negatives: 8
True Positives: 20


In [85]:
matrixplot = [
    go.Heatmap(
        z= [[matrix[1][0], matrix[1][1]],[matrix[0][0], matrix[0][1]]],
        x=['Predicted No', 'Predicted Yes'],
        y=['Actual Yes', 'Actual No'],
    )
]
py.iplot(matrixplot, filename='labelled-heatmap', title='Decision Tree Heatmap')
